In [4]:
import deepcut
from pathlib import Path
from pythainlp import word_tokenize
from matplotlib import pyplot as plt

import re
import pandas as pd
import numpy as np
from numpy import array
from numpy import argmax

from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder
from keras.preprocessing.sequence import pad_sequences
from keras.preprocessing.text import Tokenizer

from nltk import FreqDist, precision, recall, f_measure, NaiveBayesClassifier
from nltk.classify import apply_features
from nltk.classify import util
import collections, itertools
from sklearn.model_selection import KFold

In [5]:
data = pd.read_csv('train.csv')

In [6]:
'''
texts = []
labels = []
with open("C:/Users/60050204/BigData/wisesight-sentiment/kaggle-competition/train.txt",encoding="utf8") as f:
    for line in f:
        texts.append(line.strip())
        
with open("C:/Users/60050204/BigData/wisesight-sentiment/kaggle-competition/train_label.txt",encoding="utf8") as f:
    for line in f:
        labels.append(line.strip())

df = pd.DataFrame({"texts":texts,"category":labels})
df.shape
'''

'\ntexts = []\nlabels = []\nwith open("C:/Users/60050204/BigData/wisesight-sentiment/kaggle-competition/train.txt",encoding="utf8") as f:\n    for line in f:\n        texts.append(line.strip())\n        \nwith open("C:/Users/60050204/BigData/wisesight-sentiment/kaggle-competition/train_label.txt",encoding="utf8") as f:\n    for line in f:\n        labels.append(line.strip())\n\ndf = pd.DataFrame({"texts":texts,"category":labels})\ndf.shape\n'

In [7]:
data = data[data.labels != 'q']

In [8]:
data.groupby('labels').describe()

words                                                               
        count unique                                                top freq
labels                                                                      
neg      7393   7393                    สุดยอดครับรถดีมาก hrv โดนแซงแน่    1
neu     13087  13087  กินง่ายกว่าสูตรเก่าเยอะ ตอนนี้ลืมไปแล้วว่าลีโอ...    1
pos      4617   4617  Nissan X-Trail ขับขี่ในเมืองก็ประหยัด ขับขี่ตะ...    1

In [9]:
data['length'] = data['words'].apply(word_tokenize).apply(len)
data['length'].describe()

count    25097.000000
mean        25.850779
std         44.237998
min          1.000000
25%          5.000000
50%         11.000000
75%         28.000000
max        758.000000
Name: length, dtype: float64

In [10]:
data

,words,labels,length
0,ประเทศเราผลิตและส่งออกยาสูบเยอะสุดในโลกจิงป่าวคับ,neu,13
1,คะ,neu,1
2,อิเหี้ยออมทำกูอยากกินเอ็มเค,neg,9
3,😅😅😅,neu,1
4,สวัสดีวันพุธ แนน อะไรนะ,neu,8
...,...,...,...
25610,อีชาติชั้ว,neg,3
25611,อีช้างเย๊ด,neg,3
25612,อยากเอาหญิง,neg,3
25613,ทมิฬ,neg,1


In [11]:
data.sample(100)

,words,labels,length
25182,ไม่ใครเห็นค่า,neg,4
2372,สวัสดีค่ะ คุณ หากสนใจ โคโลราโด ไฮคันทรี สตอร์ม...,neu,86
49,เบียร์สปายสเมอร์นอฟเป๊บซี่ ยังไง เข้าห้องน้ำสา...,neg,15
8093,มันได้โปรแค่วันธรรมดาอ่ะเบล จันทร์-ศุกร์😂😂,neg,13
13118,ร้าน MK สาขาใกล้บ้านได้เลยค่ะ ^^,neu,12
...,...,...,...
5400,บาร์บีคิวพลาซ่า ยัน จุดสีขาวนั่นไม่ใช่พยาธิ!,neu,13
13088,อ้วนพันล้านลอรีเอะคะ55555,neu,6
4035,กุพึ่งกินมาเอง,pos,5
4871,มาปักด้วย,neu,3


In [12]:
from gensim.models import Word2Vec 

In [13]:
y_train = [row for row in data['labels']]
Y_train = array(y_train)
Y_train.shape

(25097,)

In [14]:
label_encoder = LabelEncoder()

In [15]:
integer_encoder = label_encoder.fit_transform(Y_train)

In [16]:
integer_encoder

array([1, 1, 0, ..., 0, 0, 0], dtype=int64)

In [17]:
ohe = OneHotEncoder(sparse=False)

In [18]:
onehot_encoded = ohe.fit_transform(integer_encoder.reshape(-1,1))

In [19]:
onehot_encoded

array([[0., 1., 0.],
       [0., 1., 0.],
       [1., 0., 0.],
       ...,
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.]])

In [20]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(data.words.values)

In [21]:
word_index = tokenizer.word_index

In [22]:
vocab_size = len(word_index)

In [23]:
vocab_size

87090

In [24]:
data['length'].describe()

count    25097.000000
mean        25.850779
std         44.237998
min          1.000000
25%          5.000000
50%         11.000000
75%         28.000000
max        758.000000
Name: length, dtype: float64

In [25]:
max_length = 758

In [26]:
x_train = tokenizer.texts_to_sequences(data.words.values)

In [27]:
x_train_padded = pad_sequences(x_train, maxlen=max_length ,padding='post')

In [28]:
x_train_padded.shape

(25097, 758)

In [29]:
from tensorflow.keras.layers import Dense, Embedding, Flatten, Dropout, Activation
from tensorflow.keras.models import Sequential

In [30]:
model = Sequential()
model.add(Embedding(vocab_size+1,32,input_length=max_length))
model.add(Flatten())
model.add(Dense(128,activation='relu'))
model.add(Dense(3,activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam',metrics=['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 758, 32)           2786912   
_________________________________________________________________
flatten (Flatten)            (None, 24256)             0         
_________________________________________________________________
dense (Dense)                (None, 128)               3104896   
_________________________________________________________________
dense_1 (Dense)              (None, 3)                 387       
Total params: 5,892,195
Trainable params: 5,892,195
Non-trainable params: 0
_________________________________________________________________


In [31]:
from sklearn.model_selection import train_test_split

In [32]:
x_train, x_test, y_train, y_test = train_test_split(x_train_padded, onehot_encoded, test_size=0.3, shuffle=True)

In [33]:
x_train.shape, y_train.shape, x_test.shape, y_test.shape

((17567, 758), (17567, 3), (7530, 758), (7530, 3))

In [34]:
model.fit(x_train,y_train,
         batch_size=20,
         epochs=5,
         verbose=1,
         validation_data=(x_test,y_test))

Train on 17567 samples, validate on 7530 samples
Epoch 1/5
17567/17567 [==============================] - 37s 2ms/sample - loss: 0.9902 - accuracy: 0.5200 - val_loss: 0.9564 - val_accuracy: 0.5223
Epoch 2/5
17567/17567 [==============================] - 40s 2ms/sample - loss: 0.5999 - accuracy: 0.7395 - val_loss: 1.0879 - val_accuracy: 0.5236
Epoch 3/5
17567/17567 [==============================] - 42s 2ms/sample - loss: 0.1752 - accuracy: 0.9359 - val_loss: 1.2577 - val_accuracy: 0.4944
Epoch 4/5
17567/17567 [==============================] - 40s 2ms/sample - loss: 0.0182 - accuracy: 0.9949 - val_loss: 1.3506 - val_accuracy: 0.5183
Epoch 5/5
17567/17567 [==============================] - 41s 2ms/sample - loss: 0.0101 - accuracy: 0.9968 - val_loss: 1.4238 - val_accuracy: 0.5189


In [35]:
test_loss,test_acc = model.evaluate(x_test,y_test)
print("\nTest accuracy:",test_acc)

7530/7530 [==============================] - 1s 149us/sample - loss: 1.4238 - accuracy: 0.5189

Test accuracy: 0.5188579


In [36]:
predicions = model.predict(x_test)